# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jul 14 2022 8:30AM,241811,10,0085870758,ISDIN Corporation,Released
1,Jul 14 2022 8:30AM,241811,10,0085870759,ISDIN Corporation,Released
2,Jul 14 2022 8:30AM,241811,10,0085870760,ISDIN Corporation,Released
3,Jul 14 2022 8:30AM,241811,10,0085870761,ISDIN Corporation,Released
4,Jul 14 2022 8:30AM,241811,10,0085870762,ISDIN Corporation,Released
5,Jul 14 2022 8:30AM,241811,10,0085870817,ISDIN Corporation,Released
6,Jul 14 2022 8:30AM,241811,10,0085870867,ISDIN Corporation,Released
7,Jul 14 2022 8:30AM,241809,10,0085870878,ISDIN Corporation,Released
8,Jul 14 2022 8:30AM,241811,10,0085870887,ISDIN Corporation,Released
9,Jul 14 2022 8:30AM,241809,10,0085870886,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
46,241826,Released,1
47,241827,Released,2
48,241828,Released,1
49,241829,Released,1
50,241832,Executing,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
241826,NaN,NaN,1.0
241827,NaN,NaN,2.0
241828,NaN,NaN,1.0
241829,NaN,NaN,1.0
241832,NaN,1.0,NaN


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
241723,9.0,2.0,0.0
241726,0.0,0.0,1.0
241740,0.0,1.0,0.0
241746,0.0,0.0,1.0
241747,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
241723,9,2,0
241726,0,0,1
241740,0,1,0
241746,0,0,1
241747,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,241723,9,2,0
1,241726,0,0,1
2,241740,0,1,0
3,241746,0,0,1
4,241747,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,241723,9,2,
1,241726,,,1
2,241740,,1,
3,241746,,,1
4,241747,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jul 14 2022 8:30AM,241811,10,ISDIN Corporation
7,Jul 14 2022 8:30AM,241809,10,ISDIN Corporation
14,Jul 14 2022 8:30AM,241808,10,ISDIN Corporation
19,Jul 14 2022 8:30AM,241807,10,ISDIN Corporation
44,Jul 14 2022 8:30AM,241812,10,ISDIN Corporation
80,Jul 14 2022 8:30AM,241816,10,"Methapharm, Inc."
84,Jul 14 2022 8:30AM,241825,10,"Methapharm, Inc."
85,Jul 14 2022 8:30AM,241824,10,"Methapharm, Inc."
94,Jul 14 2022 8:30AM,241826,10,"Methapharm, Inc."
95,Jul 14 2022 8:30AM,241800,10,Eywa Pharma Inc.


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jul 14 2022 8:30AM,241811,10,ISDIN Corporation,,,48
1,Jul 14 2022 8:30AM,241809,10,ISDIN Corporation,,,15
2,Jul 14 2022 8:30AM,241808,10,ISDIN Corporation,,,1
3,Jul 14 2022 8:30AM,241807,10,ISDIN Corporation,,,2
4,Jul 14 2022 8:30AM,241812,10,ISDIN Corporation,,,14
5,Jul 14 2022 8:30AM,241816,10,"Methapharm, Inc.",,,4
6,Jul 14 2022 8:30AM,241825,10,"Methapharm, Inc.",,,3
7,Jul 14 2022 8:30AM,241824,10,"Methapharm, Inc.",,,7
8,Jul 14 2022 8:30AM,241826,10,"Methapharm, Inc.",,,1
9,Jul 14 2022 8:30AM,241800,10,Eywa Pharma Inc.,,,2


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 14 2022 8:30AM,241811,10,ISDIN Corporation,48,,
1,Jul 14 2022 8:30AM,241809,10,ISDIN Corporation,15,,
2,Jul 14 2022 8:30AM,241808,10,ISDIN Corporation,1,,
3,Jul 14 2022 8:30AM,241807,10,ISDIN Corporation,2,,
4,Jul 14 2022 8:30AM,241812,10,ISDIN Corporation,14,,
5,Jul 14 2022 8:30AM,241816,10,"Methapharm, Inc.",4,,
6,Jul 14 2022 8:30AM,241825,10,"Methapharm, Inc.",3,,
7,Jul 14 2022 8:30AM,241824,10,"Methapharm, Inc.",7,,
8,Jul 14 2022 8:30AM,241826,10,"Methapharm, Inc.",1,,
9,Jul 14 2022 8:30AM,241800,10,Eywa Pharma Inc.,2,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 14 2022 8:30AM,241811,10,ISDIN Corporation,48,,
1,Jul 14 2022 8:30AM,241809,10,ISDIN Corporation,15,,
2,Jul 14 2022 8:30AM,241808,10,ISDIN Corporation,1,,
3,Jul 14 2022 8:30AM,241807,10,ISDIN Corporation,2,,
4,Jul 14 2022 8:30AM,241812,10,ISDIN Corporation,14,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 14 2022 8:30AM,241811,10,ISDIN Corporation,48.0,NaN,NaN
1,Jul 14 2022 8:30AM,241809,10,ISDIN Corporation,15.0,NaN,NaN
2,Jul 14 2022 8:30AM,241808,10,ISDIN Corporation,1.0,NaN,NaN
3,Jul 14 2022 8:30AM,241807,10,ISDIN Corporation,2.0,NaN,NaN
4,Jul 14 2022 8:30AM,241812,10,ISDIN Corporation,14.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 14 2022 8:30AM,241811,10,ISDIN Corporation,48.0,0.0,0.0
1,Jul 14 2022 8:30AM,241809,10,ISDIN Corporation,15.0,0.0,0.0
2,Jul 14 2022 8:30AM,241808,10,ISDIN Corporation,1.0,0.0,0.0
3,Jul 14 2022 8:30AM,241807,10,ISDIN Corporation,2.0,0.0,0.0
4,Jul 14 2022 8:30AM,241812,10,ISDIN Corporation,14.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,4110733,109.0,1.0,3.0
12,241771,0.0,1.0,0.0
16,2417709,7.0,6.0,18.0
19,1450743,51.0,1.0,6.0
20,967065,22.0,5.0,3.0
21,1450828,6.0,0.0,0.0
22,725325,2.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,4110733,109.0,1.0,3.0
1,12,241771,0.0,1.0,0.0
2,16,2417709,7.0,6.0,18.0
3,19,1450743,51.0,1.0,6.0
4,20,967065,22.0,5.0,3.0
5,21,1450828,6.0,0.0,0.0
6,22,725325,2.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,109.0,1.0,3.0
1,12,0.0,1.0,0.0
2,16,7.0,6.0,18.0
3,19,51.0,1.0,6.0
4,20,22.0,5.0,3.0
5,21,6.0,0.0,0.0
6,22,2.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,109.0
1,12,Released,0.0
2,16,Released,7.0
3,19,Released,51.0
4,20,Released,22.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,16,19,20,21,22
Status,,,,,,,
Completed,3.0,0.0,18.0,6.0,3.0,0.0,0.0
Executing,1.0,1.0,6.0,1.0,5.0,0.0,1.0
Released,109.0,0.0,7.0,51.0,22.0,6.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,16,19,20,21,22
0,Completed,3.0,0.0,18.0,6.0,3.0,0.0,0.0
1,Executing,1.0,1.0,6.0,1.0,5.0,0.0,1.0
2,Released,109.0,0.0,7.0,51.0,22.0,6.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,16,19,20,21,22
0,Completed,3.0,0.0,18.0,6.0,3.0,0.0,0.0
1,Executing,1.0,1.0,6.0,1.0,5.0,0.0,1.0
2,Released,109.0,0.0,7.0,51.0,22.0,6.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()